In [1]:
import re
import gensim
import logging
import pandas as pd
from gensim.models import word2vec

In [2]:
import logging
import warnings
warnings.filterwarnings('ignore')

In [5]:
m = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'

In [6]:
model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)

# Задание

1. Найдите по 5 ближайших слов к словам "город", "деревня", "спорт", "бизнес", "Россия", "происшествие", "река", "озеро", "море", "горы", "депутат", "врач", "север", "юг", "Кавказ", "Сибирь", "газпром", название своего родного города... Учтите, что слова может не быть в модели!


2. Посчитайте семантическую близкость слов "театр" и "кино", "Владивосток" и "Москва", "церковь" и "государство", "культура" и "отдых", "преступление" и "наказание". Можете придумать свои пары слов.


3. Решите примеры (можно придумать свои):
 * москва + екатеринбург - собянин
 * спартак - москва + санкт-петербург
 * иркутск - байкал + сочи
 * татарстан - татарский + бурятия
 * чай - лимон + кофе
 * авиакомпания - аэрофлот + ржд
 
 
4. Найдите лишнее, попробуйте интерпретировать результаты
 * магазин, супермаркет, рынок, тц
 * теннис, хоккей, футбол, дзюдо
 * кошка, собака, попугай, кролик
 * коми, дагестан, башкирия, камчатка

In [7]:
model.most_similar('город_S')[:5]

[('столица_S', 0.691425085067749),
 ('деревня_S', 0.646914005279541),
 ('поселок_S', 0.6450192928314209),
 ('городок_S', 0.6281070709228516),
 ('селение_S', 0.5758612155914307)]

In [8]:
model.wv.similarity("театр_S", "кино_S")

0.462896

### Подсказка

Может произойти такое, что вы запишете слово и модель его не найдет, хотя по идее оно должно там быть. Например, если вы решите записать `Санкт Петербург` как `СанктПетербург_S`. На это случай можно просто заглянуть в вокаб модели и посмотреть, есть ли там что-то похожее на нужное вам слово:

In [8]:
for w in model.wv.vocab:
    if w.startswith('санкт'):
        print(w)

санкт-петербург_S
санкт-петербургский_A
санкт_COM
санктпетербург_S
санктпетербургский_A
санкт-петербуржский_A
санктпитербурх_S
санктис_S
санктъпитербурху_ADV
санктпетерсбурка_S
санктпитербурский_A
санкт_S


## Пример решения

In [13]:
words = ['москва_S', 'екатеринбург_S', 'собянин_S', 'спартак_S', 'санкт-петербург_S', 'иркутск_S',
         'байкал_S', 'сочи_S', 'татарстан_S', 'татарский_A', 'бурятия_S',
         'чай_S', 'лимон_S', 'кофе_S', 'авиакомпания_S', 'аэрофлот_S', 'ржд_S']

In [17]:
for word in words:
    # есть ли слово в модели? 
    if word in model:
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(f'{word}\t{model[word][:10]}')
    else:
        # Увы!
        print('Увы, слова "%s" нет в модели!' % word)

москва_S	[ 0.05887863  0.04578568  0.00118581  0.05433519  0.04515051  0.0083573
  0.1119729  -0.08134489  0.05941165  0.12911801]
екатеринбург_S	[ 0.04907962  0.01020678 -0.01604161  0.03411315 -0.08218605 -0.00356975
  0.11838276  0.03056966 -0.01434953  0.14566953]
собянин_S	[ 0.03149601  0.01003106  0.02190163  0.07521112 -0.00642954  0.00192631
  0.09507941 -0.00526249  0.01127878  0.00836256]
спартак_S	[-0.00524524  0.0305879  -0.0041848   0.09794901 -0.00691661  0.04532368
 -0.00682387  0.0288958   0.03240622  0.07399928]
санкт-петербург_S	[ 0.0461253   0.04253832 -0.05203993  0.0569893  -0.1648369  -0.01363693
  0.09160395  0.06810528 -0.07754476  0.11752215]
иркутск_S	[ 0.04432662 -0.06865826  0.01330303 -0.00073656  0.00362124  0.05771408
  0.1714731  -0.02286288 -0.03303407  0.04132642]
байкал_S	[ 0.0848878  -0.0736421  -0.04214313 -0.06788941 -0.04859823  0.01735261
  0.12106284 -0.09871329 -0.12328883  0.03607318]
сочи_S	[ 0.04040097 -0.03900892  0.01079668  0.00509841  0.

Решите примеры (можно придумать свои):
 * москва + екатеринбург - собянин
 * спартак - москва + санкт-петербург
 * иркутск - байкал + сочи
 * татарстан - татарский + бурятия
 * чай - лимон + кофе
 * авиакомпания - аэрофлот + ржд

In [18]:
print(model.most_similar(positive=['москва_S', 'екатеринбург_S'], negative=['собянин_S'])[0][0])
print(model.most_similar(positive=['спартак_S', 'санкт-петербург_S'], negative=['москва_S'])[0][0])
print(model.most_similar(positive=['иркутск_S', 'сочи_S'], negative=['байкал_S'])[0][0])
print(model.most_similar(positive=['татарстан_S', 'бурятия_S'], negative=['татарский_A'])[0][0])
print(model.most_similar(positive=['чай_S', 'кофе_S'], negative=['лимон_S'])[0][0])
print(model.most_similar(positive=['авиакомпания_S', 'ржд_S'], negative=['аэрофлот_S'])[0][0])

петербург_S
цска_S
москва_S
карачаево-черкесский_A
чай_ADV
газпром_S


Найдите лишнее, попробуйте интерпретировать результаты
 * магазин, супермаркет, рынок, тц
 * теннис, хоккей, футбол, дзюдо
 * кошка, собака, попугай, кролик
 * коми, дагестан, башкирия, камчатка

In [19]:
print(model.doesnt_match('магазин, супермаркет, рынок, тц,'.replace(',', '_S').split()))
print(model.doesnt_match('теннис, хоккей, футбол, дзюдо,'.replace(',', '_S').split()))
print(model.doesnt_match('кошка, собака, попугай, кролик,'.replace(',', '_S').split()))
print(model.doesnt_match('коми, дагестан, башкирия, камчатка,'.replace(',', '_S').split()))

тц_S
дзюдо_S
попугай_S
камчатка_S


## Дополнительно

моно написать простейшую игру

In [5]:
import random

сама игра

In [7]:
def game(play_vocab):
    print('игра началась!')
    play = []
    print('Дано 4 слова:')
    for i in range(4):
        w = random.choice(play_vocab)
        play.append(w)
        print(re.sub('_.','',w))
    guess = input('Какое слово лишнее?')
    guess = guess.lower().strip()
    answer = model.doesnt_match(play)
    if guess == re.sub('_.', '', answer):
        print('Правильно!')
    else:
        print('Неправильно!')
    print('ответ: {}'.format(re.sub('_.', '', answer)))

создаем игровой словарь

In [6]:
# в данном случае я просто взяла все слова и буду выбирать из них радномно 4
play_vocab = list(model.wv.vocab.keys())

In [9]:
game(play_vocab)

игра началась!
Дано 4 слова:
homeNKN
немкоммуна
рамана
армани
Какое слово лишнее?homeNKN
Неправильно!
ответ: немкоммуна


В таком варианте игра достаточно бессмысленная, но есть заменить `play_vocab` на какой-то осмысленный список слов, то может получиться интересно.